In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'learned'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_learned_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,0,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
0,4.66,-0.0,0.00,0.0,0.00,-0.0,-0.0,0.0,-0.0,0.0,...,-0.0,-0.0,-0.0,-0.0,0.0,0.00,0.00,0.00,0.00,0.00
2,-0.00,6.1,0.00,-0.0,-0.00,-0.0,-0.0,-0.0,-0.0,-0.0,...,0.0,-0.0,0.0,0.0,0.0,-0.00,-0.00,0.00,0.00,0.00
3,0.00,0.0,2.68,0.0,-0.00,0.0,0.0,-0.0,-0.0,0.0,...,0.0,0.0,-0.0,0.0,-0.0,0.00,-0.00,-0.00,0.00,-0.00
4,0.00,-0.0,0.00,8.4,-0.00,-0.0,-0.0,-0.0,-0.0,0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.00,0.00,0.00,-0.00,0.00
5,0.00,-0.0,-0.00,-0.0,5.16,-0.0,-0.0,0.0,0.0,0.0,...,-0.0,0.0,0.0,-0.0,-0.0,0.00,0.00,0.00,0.00,-0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.00,-0.0,0.00,-0.0,0.00,0.0,-0.0,0.0,0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,6.16,-0.00,0.00,0.00,-0.00
60,0.00,-0.0,-0.00,0.0,0.00,-0.0,0.0,-0.0,-0.0,0.0,...,-0.0,0.0,-0.0,-0.0,0.0,-0.00,0.44,0.00,0.00,-0.00
61,0.00,0.0,-0.00,0.0,0.00,0.0,0.0,0.0,-0.0,0.0,...,-0.0,-0.0,0.0,0.0,0.0,0.00,0.00,1.78,0.00,-0.00
62,0.00,0.0,0.00,-0.0,0.00,-0.0,-0.0,0.0,-0.0,-0.0,...,0.0,-0.0,-0.0,-0.0,0.0,0.00,0.00,0.00,4.82,-0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,0,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
0,1.00000,-0.00001,0.00002,0.00005,0.00000,-0.00000,-0.00000,0.00003,-0.00004,0.00003,...,-0.00009,-0.00003,-0.00003,-0.00004,0.00003,0.00002,0.00004,0.00005,0.00003,0.00004
2,-0.00001,1.00000,0.00001,-0.00005,-0.00004,-0.00004,-0.00000,-0.00004,-0.00000,-0.00004,...,0.00001,-0.00004,0.00003,0.00001,0.00001,-0.00004,-0.00003,0.00002,0.00001,0.00004
3,0.00002,0.00001,1.00000,0.00003,-0.00002,0.00002,0.00003,-0.00004,-0.00002,0.00002,...,0.00002,0.00005,-0.00004,0.00001,-0.00001,0.00003,-0.00002,-0.00002,0.00003,-0.00004
4,0.00005,-0.00005,0.00003,1.00000,-0.00001,-0.00003,-0.00001,-0.00003,-0.00004,0.00000,...,-0.00001,-0.00005,-0.00000,-0.00000,-0.00001,-0.00002,0.00002,0.00003,-0.00003,0.00005
5,0.00000,-0.00004,-0.00002,-0.00001,1.00000,-0.00005,-0.00001,0.00003,0.00004,0.00003,...,-0.00001,0.00001,0.00001,-0.00002,-0.00006,0.00000,0.00004,0.00002,0.00003,-0.00002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.00002,-0.00004,0.00003,-0.00002,0.00000,0.00001,-0.00005,0.00003,0.00001,-0.00002,...,-0.00005,-0.00001,-0.00002,-0.00004,-0.00000,1.00000,-0.00002,0.00004,0.00005,-0.00000
60,0.00004,-0.00003,-0.00002,0.00002,0.00004,-0.00004,0.00005,-0.00000,-0.00002,0.00003,...,-0.00008,0.00006,-0.00000,-0.00002,0.00002,-0.00002,1.00000,0.00002,0.00003,-0.00001
61,0.00005,0.00002,-0.00002,0.00003,0.00002,0.00004,0.00003,0.00001,-0.00001,0.00000,...,-0.00002,-0.00001,0.00001,0.00006,0.00003,0.00004,0.00002,1.00000,0.00001,-0.00001
62,0.00003,0.00001,0.00003,-0.00003,0.00003,-0.00001,-0.00001,0.00002,-0.00001,-0.00000,...,0.00000,-0.00001,-0.00001,-0.00003,0.00000,0.00005,0.00003,0.00001,1.00000,-0.00001


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.007584825682511296

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.45711734e+00 4.29997836e+00 3.93592996e+00 1.14254266e+00
 1.03106474e+00 8.57247053e-01 6.43464483e-01 6.27482327e-01
 6.17715891e-01 6.00879214e-01 5.73300042e-01 5.46469136e-01
 5.20020223e-01 5.13098305e-01 4.89338933e-01 4.48388589e-01
 4.23638888e-01 3.74673648e-01 3.48749493e-01 3.21737846e-01
 2.90614635e-01 2.71258608e-01 1.80296162e-01 1.73623456e-01
 1.41326401e-01 1.36836031e-01 1.31268669e-01 1.27667046e-01
 1.21398051e-01 1.15097281e-01 1.00079478e-01 9.86978909e-02
 9.54124197e-02 9.50424158e-02 9.31911989e-02 8.99731257e-02
 8.60869225e-02 8.25306726e-02 8.09890758e-02 7.83540138e-02
 7.51876707e-02 7.48959283e-02 7.02177038e-02 6.60047504e-02
 6.33236343e-02 6.13810892e-02 6.05543120e-02 5.95592268e-02
 5.44596820e-02 5.10439661e-02 4.93875169e-02 4.80195947e-02
 4.68103976e-02 4.53934970e-02 3.94095975e-02 3.71013801e-02
 3.61633906e-02 2.83862754e-02 2.60638736e-02 5.34787025e-03
 1.68984089e-03 8.15022739e-34]

Principal comp

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,-0.003834,-0.005342,-0.002092,-0.008243,-0.004301,-0.001477,-0.002636,-0.000182,-0.001488,-0.001746,...,-0.001765,-0.001343,-0.004815,-0.001566,-0.002298,-0.005372,-0.000340,-0.001371,-0.004021,-0.000953
1,-0.004136,-0.005790,-0.002203,-0.009361,-0.004660,-0.001579,-0.002766,-0.000190,-0.001565,-0.001827,...,-0.001848,-0.001440,-0.005156,-0.001672,-0.002432,-0.005823,-0.000358,-0.001457,-0.004286,-0.001008
2,-0.006234,-0.008800,-0.003344,-0.014440,-0.007060,-0.002396,-0.004213,-0.000289,-0.002387,-0.002803,...,-0.002792,-0.002134,-0.007881,-0.002530,-0.003678,-0.008920,-0.000550,-0.002205,-0.006494,-0.001550
3,-0.009117,-0.017941,-0.004040,0.989034,-0.011370,-0.002786,-0.005316,-0.000328,-0.002780,-0.003258,...,-0.003311,-0.002510,-0.014054,-0.002917,-0.004511,-0.018479,-0.000601,-0.002519,-0.009892,-0.001712
4,-0.012161,-0.025802,-0.005175,0.119059,-0.015403,-0.003522,-0.006792,-0.000403,-0.003528,-0.004174,...,-0.004173,-0.003148,-0.019391,-0.003757,-0.005697,-0.026738,-0.000754,-0.003214,-0.013130,-0.002189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,-0.012296,-0.009077,-0.025858,-0.006469,-0.010906,-0.051294,-0.019094,0.007447,-0.051822,-0.036369,...,-0.035510,-0.070088,-0.009908,-0.045095,-0.022833,-0.008971,0.015199,-0.065462,-0.011817,0.380387
58,-0.005103,-0.003755,-0.010457,-0.002682,-0.004497,-0.020008,-0.007771,0.003400,-0.020156,-0.014421,...,-0.014151,-0.026216,-0.004080,-0.017753,-0.009259,-0.003694,0.006956,-0.024735,-0.004857,0.878879
59,-0.043344,-0.032871,-0.077759,-0.023814,-0.039028,-0.113199,-0.062343,0.174229,-0.113651,-0.095523,...,-0.094316,-0.127678,-0.035712,-0.106775,-0.071214,-0.032534,0.681660,-0.124698,-0.041833,-0.201375
60,-0.028154,-0.021458,-0.049448,-0.015578,-0.025398,-0.069766,-0.040055,0.657111,-0.069992,-0.059819,...,-0.059188,-0.077434,-0.023281,-0.066208,-0.045496,-0.021247,-0.617786,-0.075894,-0.027195,-0.111983


In [13]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine')
cos_dist = np.sort(np.concatenate(cos_dist))[:len(GROUPS)]
cos_dist


array([0.00327013, 0.00960953, 0.01096601, 0.01388111, 0.01985558,
       0.01993857, 0.05010803, 0.06556338, 0.07889078, 0.09703867,
       0.10122451, 0.10407926, 0.10410666, 0.10706078, 0.114373  ,
       0.1188519 , 0.12112103, 0.12852174, 0.12961303, 0.1328762 ,
       0.13774165, 0.14593236, 0.14700364, 0.1486396 , 0.15063551,
       0.15197482, 0.15728729, 0.16101287, 0.16870979, 0.17613428,
       0.17929069, 0.18669995, 0.19780417, 0.2056491 , 0.2163851 ,
       0.21682493, 0.21690934, 0.2171972 , 0.22284397, 0.24354388,
       0.24556453, 0.24620245, 0.25064498, 0.26499929, 0.26672472,
       0.27070053, 0.27742458, 0.27848653, 0.27878667, 0.27907454,
       0.2935346 , 0.29617615, 0.30791768, 0.31501247, 0.31834045,
       0.32799686, 0.34288946, 0.37154874, 0.39682656, 0.43824667,
       0.45592691, 0.47522233])